In [7]:
import pandas as pd
import numpy as np
df = pd.read_excel("/Users/snow/Documents/IRON HACK COURSE/Week 2/Mini Project/Week-2-Mini-Project---Shark-Attacks/GSAF5.xls")
df["Time"].unique()

array(['0930hrs', '1524hrs', 'Not stated', '0730hrs', '1300hrs',
       '1055 hrs', '1630hrs', '1130hrs', 'PM', '1150hrs', '1500hrs',
       '1600', '1615 hrs', '1210hrs', '1200hrs', '1211 hrs', '1400hrs',
       '1100hrs', 'pm', '0945hrs', '1430hrs', '1210 hrs', '1340hrs',
       'Unknown', '0815hrs', '1503hrs', '1830hrs', '1645 hrs', '1711hrs',
       '1600hrs', '1330hrs', '1615hr', '1710hr', '?', '1637hr', 'AM',
       '1600hr', '1735hrs', '1115hrs', '1118hrs', '1615hrs', '1100hr',
       'after 1200hr', '1400hr', 15.5, '13h15', '9h', 1300, '14h',
       '15h30', '13h30', '9h15', 'Not advised', '13h40', '12h30', '16h00',
       nan, '11h30', '06h30', '20h00', '13h00', '11h12', '16h30', '15h00',
       '02h00', '09h15', 'Early Morning', '16h32', '11h00', 'Morning',
       '10h30', '13h20', '14h00', '09h00', '10h20', '15h05', '17h00',
       '15h45', '07h45', '10h40', '07h50', '01h00', '10h00', 'Afternoon',
       '19h30', 'Evening', '17h50', '09h30', '08h45', '"Midday"', '16h25',
   

In [9]:
# %pip install xlrd
import pandas as pd
import numpy as np
import re

df = pd.read_excel("/Users/snow/Documents/IRON HACK COURSE/Week 2/Mini Project/Week-2-Mini-Project---Shark-Attacks/GSAF5.xls")

# Now we want to clean the dataframe a bit
# First lets have a look at the Dataframe
print(df.shape, '\n')
print(df.nunique(), '\n')

# Then we want to standardize the column names
df.columns = df.columns.str.lower().str.replace(" ","_").str.replace(":", "")
df.rename(columns={"species_":"species"}, inplace=True)

# We can see that there are some weird values in the 'Type' column such as 'Boat', 'Questionable ', 'Unconfirmed'
# We want to merge these values into a single category called 'Under investigation'

# First, we drop the rows where 'Type' and 'Sex' are null
df.dropna(subset=["sex", "type"], inplace=True)

# Then, we replace the values in the 'Type' column
def replace_type(value):
    if isinstance(value, str):
        # Remove white spaces and compare lower-case
        value_clean = value.strip().lower()
        valid_types = ['unprovoked', 'provoked']
        if value_clean not in valid_types:
            return "Under investigation"
        else:
            # Return the standardized value (capitalize first letter)
            return value_clean.capitalize()
    return value

df["type"] = df["type"].apply(replace_type)

# Now we can check the value counts again
print(df["type"].value_counts(), '\n')

# Now we want to clean the 'sex' column in the same way
def replace_sex(value):
    if isinstance(value, str):
        if value[0].lower() == "f":
            return "F"
        else:
            return "M"

df["sex"] = df["sex"].apply(replace_sex)
print(df["sex"].value_counts(), '\n')
print(df['sex'].unique(), '\n')

# Now lets dive into the 'time' column and clean it
# We want to have four unique values with the following criteria:
# 6 am until noon -> Morning
# Noon until 6 pm -> Afternoon
# 6 pm until midnight -> Night
# Midnight until 6 am -> Dawn

df['time'].unique()
#df['time'].value_counts()

# We need to use regex to extract the hour and minutes from the 'time' column
# We will create a function to clean the 'time' column
def time_cleaner(val):
    if pd.isna(val):
        return np.nan # Return NaN for null values
    
    val = str(val).strip().lower() # Convert to string and lower case
    
    # Remove unwanted characters
    val = re.sub(r'[^0-9a-z: ]', '', val) # Keep only alphanumeric characters, colon, and space

    # The previous regex cleans the column to keep only the essential characters needed for time parsing
    # The ^ inside []indicates negation, means "match everything EXCEPT these characters", so we basically removing unwanted punctuation, uppercase letters, and special characters while preserving the useful parts

    # Simple textual cases using any() for efficiency
    if any(word in val for word in ['dawn', 'daybreak', 'sunrise', 'early morning']):
        return 'Dawn'
    if any(word in val for word in ['morning']):
        return 'Morning'
    if any(word in val for word in ['noon', 'midday', 'afternoon', 'after noon', 'mid afternoon', 'late afternoon']):
        return 'Afternoon'
    if any(word in val for word in ['evening', 'dusk', 'sunset', 'night', 'dark']):
        return 'Evening'

    # Numeric time formats using regex to capture hour and optional minutes with or without separators
    # This regex is designed to extract time information from various formats
    match = re.search(r'(\d{1,2})[:h]?(\d{0,2})?', val)
    if match:
        hour = int(match.group(1)) # Extract hour from regex group 1
        minutes = match.group(2) if match.group(2) else '00' # Extract minutes from regex group 2 if available

        # 4 digits format without separator (ex: 1530)
        if len(val) >= 4 and val[:4].isdigit(): # Check if first 4 characters are digits
            hour = int(val[:2]) # First two digits are hour
            minutes = val[2:4] # Last two digits are minutes

        # Adjust hour if minutes are superior to 45
        if minutes and minutes.isdigit() and int(minutes) > 45: # If minutes are greater than 45, round up the hour
            hour += 1
        hour = hour % 24  # Ensure hour is within 0-23

        # Convert in 4 periods of the day
        if 5 <= hour < 11:
            return 'Morning'
        elif 11 <= hour < 18:
            return 'Afternoon'
        elif 18 <= hour < 23:
            return 'Evening'
        else:
            return 'Dawn'

    # If nothing matches, return NaN
    return np.nan

# We apply the function to the 'time' column
df['time_clean'] = df['time'].apply(time_cleaner)

# To check the result
print(df['time_clean'].value_counts(dropna=False))

# # Now we want to drop the columns that are not useful for our analysis
df = df.drop(df.columns[[7, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]], axis=1, inplace=False)
# # We are droping time column because we created a new column 'time_clean' with the cleaned values
# # We did drop the name column because of the data privacy policy in some countries
df.head()






(7045, 23) 

Date              6089
Year               261
Type                13
Country            247
State              937
Location          4595
Activity          1606
Name              5770
Sex                 10
Age                250
Injury            4162
Fatal Y/N           12
Time               460
Species           1724
Source            5384
pdf               6789
href formula      6784
href              6776
Case Number       6777
Case Number.1     6775
original order    6797
Unnamed: 21          1
Unnamed: 22          2
dtype: int64 

type
Unprovoked             5032
Under investigation     834
Provoked                582
Name: count, dtype: int64 

sex
M    5645
F     803
Name: count, dtype: int64 

['M' 'F'] 

time_clean
NaN          3128
Afternoon    2105
Morning       766
Evening       394
Dawn           55
Name: count, dtype: int64


,date,year,type,country,state,location,activity,sex,age,injury,fatal_y/n,time_clean
0,6th September,2025.0,Unprovoked,Australia,NSW,Long Reef Sydney,Surfing,M,57,Both legs and arm severed,Y,Morning
1,1st September,2025.0,Unprovoked,USA,Florida,Horseshoe reef Key Largo,Snorkeling,M,8,Bite to leg,N,Afternoon
2,30th August,2025.0,Unprovoked,USA,Texas,Galveston,Swimming,F,8,Bite to leg,N,NaN
3,18th August,2025.0,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,M,?,None sustained board severly damaged,N,Morning
4,17th August,2025.0,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,M,63,Severe injuries no detail,N,Afternoon


In [21]:
df.isna().sum()
df[['time_clean']].describe()

,time_clean
count,3320
unique,4
top,Afternoon
freq,1581


In [14]:
display(df[['type']].describe())

,type
count,6448
unique,3
top,Unprovoked
freq,5032


In [15]:
display(df[['sex']].describe())

,sex
count,6448
unique,2
top,M
freq,5645
